## Fixed Random Seed

In [2]:
import numpy as np
import random
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## 데이터 불러오기 및 확인

In [3]:
import pandas as pd

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

display(train.head(3))
# display(test.head(3))

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,...,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860


## 데이터 시각화 : 근데 이제 Seaborn을 곁들인

## 데이터 전처리 1 : 학습 및 추론 데이터 설정

In [4]:
train_x = train.drop(columns=['ID', 'Income'])
train_y = train['Income']

test_x = test.drop(columns=['ID'])

## 데이터 전처리 2 : 범주형 변수 수치화

In [5]:
train_x.dtypes

Age                        int64
Gender                    object
Education_Status          object
Employment_Status         object
Working_Week (Yearly)      int64
Industry_Status           object
Occupation_Status         object
Race                      object
Hispanic_Origin           object
Martial_Status            object
Household_Status          object
Household_Summary         object
Citizenship               object
Birth_Country             object
Birth_Country (Father)    object
Birth_Country (Mother)    object
Tax_Status                object
Gains                      int64
Losses                     int64
Dividends                  int64
Income_Status             object
dtype: object

In [6]:
from sklearn.preprocessing import LabelEncoder

# data type 이 object인 데이터만 리스트에 담기(반복문에 넣기 위해서)
encoding_target = list(train_x.dtypes[train_x.dtypes == "object"].index)

# 각 칼럼마다 레이블 인코딩 적용
for i in encoding_target:
    le = LabelEncoder()
    
    # train과 test 데이터셋에서 해당 열의 모든 값을 문자열로 변환
    train_x[i] = train_x[i].astype(str)
    test_x[i] = test_x[i].astype(str)

    # 반드시 trian에만 fit 하고 test에는 fit하면 안됨(이유 궁금하면 찾아보세요)
    le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    # le.classes_에서 train에는 없지만 test 에만 있는 값이 있으면 le.classes_에 해당 값을 추가
    for case in np.unique(test_x[i]):
        if case not in le.classes_: 
            le.classes_ = np.append(le.classes_, case)
    test_x[i] = le.transform(test_x[i])


## 데이터 전처리 3 : 자유롭게 전처리 해보세요.

## 모델 선정 및 학습

In [7]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor() 
model.fit(train_x, train_y) 

DecisionTreeRegressor()

In [ ]:
# DecisionTreeRegressor로 mae, mse, rmse를 모두 구해보세요.

In [ ]:
# train set을 '선형회귀 모델'로 학습한 후 validation set의 rmse를 계산하세요.

In [25]:
# train set을 '다항회귀 모델'로 학습한 후 validation set의 rmse를 계산하세요.

In [26]:
# train set을 'L1 regularization 모델(라쏘)'로 학습한 후 validation set의 rmse를 계산하세요.

In [27]:
# train set을 'L2 regularization 모델(릿지)'로 학습한 후 validation set의 rmse를 계산하세요.

In [ ]:
# 그동안 학습했던 모든 것들을 동원해 다양한 모델들을 시험해보세요.

## 하이퍼 파라미터 최적화

In [24]:
# 여러 모델 중 임의의 모델을 하나 선정해서 Optuna를 통해 하이파 파라미터 최적화를 해보세요.

## 앙상블

In [28]:
# Optuna를 통해 도출된 최적화된 모델을 앙상블 기법 중 부스팅을 이용해서 성능을 올려보세요.

In [ ]:
# 자유롭게 앙상블 해보세요.(보팅, 배깅, 부스팅 등)

## 예측 수행

In [6]:
preds = model.predict(test_x)

## 제출양식에 예측결과 입력

In [7]:
submission = pd.read_csv('./sample_submission.csv')
submission['Income'] = preds
submission

,ID,Income
0,TEST_0000,0.0
1,TEST_0001,0.0
2,TEST_0002,500.0
3,TEST_0003,0.0
4,TEST_0004,0.0
...,...,...
9995,TEST_9995,1300.0
9996,TEST_9996,700.0
9997,TEST_9997,425.0
9998,TEST_9998,0.0


## 예측결과 저장

In [8]:
submission.to_csv('./baseline_submission.csv', index=False)